<a href="https://colab.research.google.com/github/jish1398/Nlu_Intent/blob/master/nlu_intents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config && pip install pygraphviz;
!pip install rasa_nlu[tensorflow];
!python -m spacy download en;
!pip install git+https://github.com/mit-nlp/MITIE.git
!pip install rasa_nlu[mitie]

Extracting templates from packages: 100%
Selecting previously unselected package fontconfig.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.12.6-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.12.6-0ubuntu2) ...
Selecting previously unselected package libxinerama1:amd64.
Preparing to unpack .../01-libxinerama1_2%3a1.1.3-1_amd64.deb ...
Unpacking libxinerama1:amd64 (2:1.1.3-1) ...
Selecting previously unselected package shared-mime-info.
Preparing to unpack .../02-shared-mime-info_1.9-2_amd64.deb ...
Unpacking shared-mime-info (1.9-2) ...
Selecting previously unselected package hicolor-icon-theme.
Preparing to unpack .../03-hicolor-icon-theme_0.17-2_all.deb ...
Unpacking hicolor-icon-theme (0.17-2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../04-libjbig0_2.1-3.1build1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1build1) ...
Selecting previously unselected package libtiff5:amd64.
Prep

Create the database and try different 

In [0]:
nlu_md = """

## intent:button
- please click(Action) the [red](color) button
- click button 
- click the [blue](color) button
- click the [blue](color) colored button
- click the [red](color) colored button
- click the [black](color) button 
- click the [first](position) button
- click the [third](position) [red](color) button
- click the [second](position) button
- press the [blue](color) button
- Please go to(Action) the next page by clicking(Action) the [red](color) button
- Press [red](color) button
- Click [blue](color) button
- Please click the [red](color) button
- Click the [first](position) [blue](color) button
- click the [first](position) [red](color) button
- clicke the[first](position) [green](color) button
- click the [second](position) [red](color) button
- click the [second](position) [blue](color) button
- click the [second](position) [green](color) button
- click the [third](position)  [red](color) button
- click the [fourth](position) [red](color) button
- click the [third](position) [blue](color) button

## lookup:Action   <!-- lookup table list -->
- press
- go to
- click
- clicking
##

## intent: Language
- change the language to [english](lang)
- change the language to [spanish](lang)
- change the language to [french](lang)   <!-- Try to add synonyms -->
- change the language to [hindi](lang)
- please change the language to [spanish](lang)
- please change the language to [english](lang)
- can you please the language to [english](lang)?
- can you please the language to [hindi](lang)?
- can you please the language to [french](lang)?
- can you please the language to [spanish](lang)?
- change language  <!-- Default Language is English -->


## intent: Navigation
- [go](action) [back](direction) 
- [go](action) [backward](direction) 
- [go](action) [forward](direction)
- [go](action) to [previous](direction) website
- [go](action) to [pr](direction) website

- [open](action) [new](type) [tab](wintab) <-- Paramters from this will be the type,action,direction,wintab -->
- [open](action) [new](type) [window](wintab) <!-- Default Language is New tab -->
- can you please go forward?
- can you please go back?
- please go forward.
- please open new tab
- please open new window
- please close the first tab
- please close the second tab
- please close the window
- please open a window 
- please open a tab

"""
%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [0]:
nlu_config = """
language: en
pipeline:
- name: "tokenizer_whitespace"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
  intent_tokenization_flag: true
  intent_split_symbol: "+"
  """
%store nlu_config > nlu_config.yml


Writing 'nlu_config' (str) to file 'nlu_config.yml'.


In [0]:
!python -m rasa_nlu.train -c nlu_config.yml --data nlu.md -o models --fixed_model_name nlu --project current --verbose

2018-11-05 09:58:02 INFO     rasa_nlu.training_data.loading  - Training data format of nlu.md is md
2018-11-05 09:58:02 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 51 (3 distinct intents)
	- Found intents: 'Language', 'button', 'Navigation'
	- entity examples: 39 (7 distinct entities)
	- found entities: 'direction', 'position', 'lang', 'type', 'wintab', 'action', 'color'

2018-11-05 09:58:02 INFO     rasa_nlu.model  - Starting to train component tokenizer_whitespace
2018-11-05 09:58:02 INFO     rasa_nlu.model  - Finished training component.
2018-11-05 09:58:02 INFO     rasa_nlu.model  - Starting to train component ner_crf
2018-11-05 09:58:02 WARNING  rasa_nlu.extractors.crf_entity_extractor  - Misaligned entity annotation in sentence 'clicke thefirst green button'. Make sure the start and end values of the annotated training examples end at token boundaries (e.g. don't include trailing whitespaces or punctuation).
2018-11-05 09:58:02 INFO 

In [0]:
from rasa_nlu.model import Interpreter
import json
interpreter = Interpreter.load("./models/current/nlu")
message = "click on the red colored button"
result = interpreter.parse(message)
print(json.dumps(result, indent=2))

/usr/local/lib/python3.6/dist-packages/rasa_nlu/extractors/entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/current/nlu/entity_synonyms.json'
  "".format(entity_synonyms_file))


INFO:tensorflow:Restoring parameters from ./models/current/nlu/intent_classifier_tensorflow_embedding.ckpt
{
  "intent": {
    "name": "button",
    "confidence": 0.970398485660553
  },
  "entities": [
    {
      "start": 13,
      "end": 16,
      "value": "red",
      "entity": "color",
      "confidence": 0.8745525813436467,
      "extractor": "ner_crf"
    }
  ],
  "intent_ranking": [
    {
      "name": "button",
      "confidence": 0.970398485660553
    },
    {
      "name": "Language",
      "confidence": 0.0
    },
    {
      "name": "Navigation",
      "confidence": 0.0
    }
  ],
  "text": "click on the red colored button"
}
